In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [ ]:
engine = create_engine() #Здесь прописан запрос для соединения с сервером через логин и пароль
conn = engine.connect().execution_options(stream_results=True)

# Скачиваем данные с сервера. Таблицу постов, пользователей и их актвность

In [ ]:
post = pd.read_sql('SELECT * FROM public.post_text_df', con=conn) # Таблица постов 

user = pd.read_sql('SELECT * FROM public.user_data', con=conn) # Таблица пользователей


#Для всех таблиц с активностью выбираем где действие отлично от like. Данная таблица уже имеет разметку
# с колонкой target которая обозначает поставил ли лайк пользователь этому посту при просмотре. 
# И если не убрать строчки с лайками, то данные будут дублироваться. 
action_lim1kk = pd.read_sql((f"SELECT * FROM public.feed_data where action != 'like' limit 1000000")
            , con=conn)

df_tar_40000 = pd.read_sql((f"SELECT * FROM public.feed_data where action != 'like' and target=1 limit 40000")
            , con=conn) # Таблица с 40к таргет

tup = tuple(user['user_id'].unique()[:2000])

us_action_2000 = pd.read_sql(f'''
        SELECT * FROM public.feed_data where user_id in {tup} and action != 'like'
''', con=conn) #Таблица со всеми действиями первых 2к пользователей


us_tar_12_24 = pd.read_sql('''
        SELECT * FROM public.feed_data 
        WHERE action != 'like' and target=1 and timestamp>'2021-12-24' and timestamp<'2021-12-29'      
''', con=conn) #Таблица со всеми действиями пользователей в период с 24 декабря по 29


#Сохраняем все таблицы для дальнейшего использования
user.to_csv('user.csv')
post.to_csv('post.csv')
action_lim1kk.to_csv('action_lim1kk.csv')
df_tar_40000.to.csv('df_tar_40000.csv', index=False)
us_action_2000.to_csv('us_action_2000.csv', sep=';', index=False)
us_tar_12_24.to_csv('us_tar_12_24.csv', sep=';', index=False)